In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# %pip install openpyxl
# %pip install wandb

In [250]:
# load excel file data
# df = pd.read_excel("~/data/project/pMHC-TCR/seqData/230213_info.xlsx")
file_path = "~/data/project/data/seqData/230215.xlsx"
df = pd.read_excel(file_path)

In [251]:
df["nSeqHLA"].value_counts()

ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAGGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATGTGAAGGCCCAGTCACAGACTGACCGAGTGGACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTTCTCACACCATCCAGATAATGTATGGCTGCGACGTGGGGCCGGACGGGCGCTTACTCCGCGGGTACCGGCAGGACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCTTGGACCGCGGCGGACATGGCAGCTCAGATCACCAAGCGCAAGTGGGAGGCGGCCCATGCGGCGGAGCAGCAGAGAGCCTACCTGGAGGGCCGGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCACGGACCCCCCCAAGACACATATGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGACCAGACCCAGGACACGGAGCTCGTGGAGACCAGGCCTGCAGGGGATGGAACCTTCCAGAAGTGGGCGGCTGTGGTGGTGCCTTCTGGAGAGGAGCAGAGATACACCTGCCATGTGCAGCATGAGGGTCTGCCCAAGCCCCTCACCCTGAGATGGGAGCTGTCTTCCCAGCCCACCATCCCCATCGTGGGCATCATTGCTGGCCTGGTTCTCCTTGGAGCTGTGATCACTGGAGCTGTGGTCGCTGCCGTGATGTGGAGGA

In [256]:
# find the longest length of aaSeqCDR1
for i in ["TRA", "TRB"]:
    for seq in ["aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]:
        print(i, seq, df.loc[df["chain"] == i, seq].str.len().max())
    # df.loc[df['chain'] == 'TRA', 'aaSeqCDR1'].str.len().max()

TRA aaSeqCDR1 7
TRA aaSeqCDR2 8
TRA aaSeqCDR3 25
TRB aaSeqCDR1 6
TRB aaSeqCDR2 7
TRB aaSeqCDR3 21


In [150]:
# reindex the df by cellname and chain
# df.sort_values(by=['cellname', "chain"])
df = df.set_index(['cellname', "chain"])
# extract the NeoAA, HLA, and aaSeqCDR columns
df = df[["NeoAA", "HLA", "aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]]
# df["aaSeqCDR1"].value_counts()
df["aaSeqCDR"] = df[df.columns[2:]].apply(
    # for TRA aaSeqCDR1, padding with 'X' to make it 7 aa long
    lambda x: x[0] + 'X' * (7 - len(x[0])) + x[1] + x[2],
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)

In [151]:
idx = pd.IndexSlice

df_a = df.loc[idx[:,"TRA"],]
df_a["AseqCDR"] = df_a["aaSeqCDR"]
df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_a.index = df_a.index.droplevel(1)
# print(df_a)
df_b = df.loc[idx[:,"TRB"],]
df_b["BseqCDR"] = df_b["aaSeqCDR"]
df_b.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_b.index = df_b.index.droplevel(1)
# print(df_b)

# merge the TRA and TRB dataframes by cellname, HLAs, and NeoAA
df_ab = pd.merge(df_a, df_b, on=["cellname", "HLA", "NeoAA"])
df_ab
# df_ab = pd.merge(df_a, df_b, on="cellname")
# df_ab

/tmp/ipykernel_4163421/3989028287.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["AseqCDR"] = df_a["aaSeqCDR"]
/tmp/ipykernel_4163421/3989028287.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
/tmp/ipykernel_4163421/3989028287.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


In [152]:
df_ab.to_csv(file_path.replace(".xlsx", ".csv"))

In [85]:
# not used
# (df['aaSeqCDR'].value_counts() > 1).sum()
count = 0
for cell in df.index:
    rep_dict = {}
    if df.loc[(cell), "aaSeqCDR"] in rep_dict:
        count += 1
    else:
        rep_dict[df.loc[(cell), "aaSeqCDR"]] = 1
print(rep_dict)
print(count)

{'SGHTA,FQGNSA,CASSLGVYEQYF': 1}
0


In [47]:
# not used
# sorting data: combine the sequence from the same cell (has the same cellname), 
# add the prefix of the column chain to the columns CDR1, CDR2, CDR3
data = df.groupby("cellname").agg(
    {
        "NeoGeneID": "first",
        "NeoAA" : "first",
        "HLA": "first",
        "chain": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR1": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR2": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR3": lambda x: x.iloc[0] + " " + x.iloc[1],
    }
)
# split each aaSeqCDR1 into 2 columns, one for A chain and one for B chain
data[['aaSeqCDR1_B', 'aaSeqCDR1_A']] = data.aaSeqCDR1.str.split(" ",expand=True,)
data[['aaSeqCDR2_B', 'aaSeqCDR2_A']] = data.aaSeqCDR2.str.split(" ",expand=True,)
data[['aaSeqCDR3_B', 'aaSeqCDR3_A']] = data.aaSeqCDR3.str.split(" ",expand=True,)
data.drop(columns=['aaSeqCDR1', 'aaSeqCDR2', 'aaSeqCDR3'], inplace=True)

In [135]:
# not used
df.groupby(['aaSeqCDR', 'chain', 'cellname']).agg(
    {
        # find the TRA and TRB chains of the same cell has the same aaSeqCDR
        'aaSeqCDR': 'count',
    }
)

In [153]:
df_ab

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


# Load the dataset into torch dataset

In [191]:
file_path = "~/data/project/pMHC-TCR/seqData/230215.csv"
datasets = pd.read_csv(file_path)
# combine the AseqCDR and BseqCDR into one column with _ as the separator
datasets["seqCDR"] = datasets["AseqCDR"] + "_" + datasets["BseqCDR"]
# drop the AseqCDR and BseqCDR columns
datasets.drop(columns=["AseqCDR", "BseqCDR"], inplace=True)
datasets.index = datasets["cellname"]
datasets.drop(columns=["cellname"], inplace=True)
datasets

,NeoAA,HLA,seqCDR
cellname,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF_SGHTA_FQGNSA_...
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF_LGHNT_FRNRAP_CAS...
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF_KGHSH_LQKENI_C...
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF_DFQATT_SNEGSKA_CS...
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF_MNHEY_SMNVEV_CASSL...
...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF_SGHAT_FQNNGV_CA...
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF_SGHNS_FNNNVP_CAS...
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF_PRHDT_FYEKMQ_CASSP...


In [192]:
# select the first 3-peptide and last 3-peptide of NeoAA sequence
# datasets["first_3"] = datasets["NeoAA"].str.slice(0,3)
# datasets["last_3"] = datasets["NeoAA"].str.slice(-4,-1)
datasets["NeoAA"] = datasets["NeoAA"].str.slice(0,3) + "_" + datasets["NeoAA"].str.slice(-4,-1)

In [194]:
datasets["NeoAA"].value_counts()

SSC_MNQ    421
VVG_GVG    167
ATA_LSG    128
VLL_LSY     64
VVV_DVG     40
VVG_DVG     21
TTA_LSG     14
VVV_GVG     14
SLM_IPH      8
LVT_LLT      8
SVC_ILS      1
Name: NeoAA, dtype: int64

We check the first 3-peptide and last 3-peptide of the neoantigen sequence, there are only 11 cases. So I think that they could be determined as one-hot encode.

In [227]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = datasets.drop("HLA", axis=1).values
y = datasets.iloc[:,1].values

labelencoder_x = LabelEncoder()
X[:,0] = labelencoder_x.fit_transform(X[:,0])
onehotEncoder = OneHotEncoder()
X_feature = onehotEncoder.fit_transform(X[:,0].reshape(-1,1)).toarray()
X_feature.shape, X[:,1].shape
X = np.hstack((X_feature, X[:,1].reshape(-1,1)))

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

### Loading the Atchley factor

In [240]:
af = pd.read_excel("~/data/project/pMHC-TCR/library/Atchley_factors.xlsx")
af.to_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv", index=False)

In [18]:
af = pd.read_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv")
af.index = af["Amino acid"]
af.drop(columns=["Amino acid"], inplace=True)
af

,Factor I,Factor II,Factor III,Factor IV,Factor V
Amino acid,,,,,
A,-0.591,-1.302,-0.733,1.570,-0.146
C,-1.343,0.465,-0.862,-1.020,-0.255
D,1.050,0.302,-3.656,-0.259,-3.242
E,1.357,-1.453,1.477,0.113,-0.837
F,-1.006,-0.590,1.891,-0.397,0.412
G,-0.384,1.652,1.330,1.045,2.064
H,0.336,-0.417,-1.673,-1.474,-0.078
I,-1.239,-0.547,2.131,0.393,0.816
K,1.831,-0.561,0.533,-0.277,1.648


In [25]:
# Use the Atchley factors to encode the aaSeqCDR, and use t to represent the _ in the sequence
# X[:,-1][0]
# 'TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF_SGHTA_FQGNSA_CASSLGVYEQYF'

def encode_seqCDR(seqCDR):
    encoding_list = []
    for i in range(len(seqCDR)):
        if seqCDR[i] == "*":
            encoding_list.append(np.zeros(5).reshape(1,5))
        else:
            encoding_list.append(af.loc[seqCDR[i]].values.reshape(1,5))
    return np.array(encoding_list).reshape(1,-1)

In [26]:
print(encode_seqCDR("WWWW***"))

[[-0.595  0.009  0.672 -2.128 -0.184 -0.595  0.009  0.672 -2.128 -0.184
  -0.595  0.009  0.672 -2.128 -0.184 -0.595  0.009  0.672 -2.128 -0.184
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.   ]]


In [310]:
# encode_seqCDR(X[:,-1][0]).shape
encode_seq_result = dict()
for i in range(X.shape[0]):
    input_seq = X[i, -1].split("_")
    # store the encoding result of each aaSeqCDR
    encode_region_result = list()
    for seq in input_seq:
        encode_region_result.append(encode_seqCDR(seq))
    encode_seq_result[i] = encode_region_result

/tmp/ipykernel_4163421/2654203887.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(encoding_list)


In [331]:
# encode_seq_result[0]
# encode_seq_result[0][0].shape, encode_seq_result[0][1].shape
# TRA aaSeqCDR1 7
# TRA aaSeqCDR2 8
# TRA aaSeqCDR3 25
# TRB aaSeqCDR1 6
# TRB aaSeqCDR2 7
# TRB aaSeqCDR3 21
for i in range(len(encode_seq_result)):
    seq_encode = encode_seq_result[i]

def collate_fn(train_data):
    train_data.sort(key = lambda data: len(data), reverse=True)

((7, 5), (8, 5))

### Split into train set and test set

In [236]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [332]:
# X_train.shape, X_test.shape
datasets

,cellname,NeoAA,HLA,AseqCDR,BseqCDR
0,V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
1,V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
2,V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
3,V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
4,V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...,...
881,V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
882,V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
883,V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF
884,V350110758_L02_531,VVVGAGDVGK,HLA-A*11:01,DSAIYN_IQSSQRE_CAVRGNNNARLMF,GTSNPN_SVGIG_CAWSPGRSPLHF


### Reload the dataset

In [2]:
# /home/wuxinchao/data/project/data/seqData/230215.csv
file_path = "/home/wuxinchao/data/project/data/seqData/230215.csv"
df = pd.read_csv(file_path)

In [11]:
# df.loc[:,["AseqCDR", "BseqCDR", "NeoAA", "HLA"]]
df["Neo"] = df["NeoAA"].str.slice(0,3) + "_" + df["NeoAA"].str.slice(-4,-1)
df.drop(columns=["NeoAA"], inplace=True)
for chain in ["AseqCDR", "BseqCDR"]:
    df[chain+"_1"] = df[chain].str.split("_").str[0]
    df[chain+"_2"] = df[chain].str.split("_").str[1]
    df[chain+"_3"] = df[chain].str.split("_").str[2]
    df.drop(columns=[chain], inplace=True)
    # df[chain] = df[chain].str.replace("*", "")
df

,cellname,HLA,Neo,AseqCDR_1,AseqCDR_2,AseqCDR_3,BseqCDR_1,BseqCDR_2,BseqCDR_3
0,V350085868_L01_502,HLA-A*02:01,VLL_LSY,TSESDYY,QEAYKQQN,CAYRAYMEYGNKLVF,SGHTA,FQGNSA,CASSLGVYEQYF
1,V350085868_L01_505,HLA-A*02:01,VLL_LSY,NIATNDY,GYKTK,CLVGTNSNSGYALNF,LGHNT,FRNRAP,CASGSPDRFEQYF
2,V350085868_L01_506,HLA-A*02:01,VLL_LSY,ATGYPS,ATKADDK,CALTVSYGGSQGNLIF,KGHSH,LQKENI,CASSPFSIGQGLTNNEKLFF
3,V350085868_L01_507,HLA-A*02:01,SLM_IPH,TSGFNG,NVLDGL,CAVKMNTGFQKLVF,DFQATT,SNEGSKA,CSAKLRGSNQPQHF
4,V350085868_L01_509,HLA-A*02:01,VLL_LSY,MNHNS,SASEGT,CASTLRAGWDEQFF,MNHEY,SMNVEV,CASSLSPGGRSEAFF
...,...,...,...,...,...,...,...,...,...
881,V350110758_L02_515,HLA-A*11:01,VVV_DVG,NSMFDY,ISSIKDK,CAAAYYTGANSKLTF,SGHAT,FQNNGV,CASSSPPGVGNEQFF
882,V350110758_L02_522,HLA-A*11:01,VVV_DVG,NSMFDY,ISSIKDK,CAASGTGTASKLTF,SGHNS,FNNNVP,CASETSNNEQFF
883,V350110758_L02_523,HLA-A*11:01,VVV_DVG,SSNFYA,MTLNGDE,CAFYGSARQLTF,PRHDT,FYEKMQ,CASSPPTTAANYGYTF
884,V350110758_L02_531,HLA-A*11:01,VVV_DVG,DSAIYN,IQSSQRE,CAVRGNNNARLMF,GTSNPN,SVGIG,CAWSPGRSPLHF


In [20]:
# Find the longest sequence in each CDR
len_map = df[["AseqCDR_1", "AseqCDR_2", "AseqCDR_3", "BseqCDR_1", "BseqCDR_2", "BseqCDR_3"]].applymap(len).max()
for column in ["AseqCDR_1", "AseqCDR_2", "AseqCDR_3", "BseqCDR_1", "BseqCDR_2", "BseqCDR_3"]:
    df[column] = df[column].str.ljust(len_map[column], "*")

In [22]:
dataset = df[['AseqCDR_1', 'AseqCDR_2', 'AseqCDR_3', 'BseqCDR_1', 'BseqCDR_2', 'BseqCDR_3', 'Neo', 'HLA']]

In [28]:
seq_list = ['AseqCDR_1', 'AseqCDR_2', 'AseqCDR_3', 'BseqCDR_1', 'BseqCDR_2', 'BseqCDR_3']
for seq in seq_list:
    encode_seq_result = list()
    for i in dataset[seq]:
        encode_seq_result.append(encode_seqCDR(i))
    col_name = seq + "_encode"
    dataset[col_name] = encode_seq_result
dataset

,AseqCDR_1,AseqCDR_2,AseqCDR_3,BseqCDR_1,BseqCDR_2,BseqCDR_3,Neo,HLA,AseqCDR_1_encode,AseqCDR_2_encode,AseqCDR_3_encode,BseqCDR_1_encode,BseqCDR_2_encode,BseqCDR_3_encode
0,TSESDYY,QEAYKQQN,CAYRAYMEYGNKLVF*****,SGHTA*,FQGNSA*,CASSLGVYEQYF*********,VLL_LSY,HLA-A*02:01,"[[-0.032, 0.326, 2.213, 0.908, 1.313, -0.228, ...","[[0.931, -0.179, -3.005, -0.503, -1.853, 1.357...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.384, ...","[[-1.006, -0.59, 1.891, -0.397, 0.412, 0.931, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
1,NIATNDY,GYKTK***,CLVGTNSNSGYALNF*****,LGHNT*,FRNRAP*,CASGSPDRFEQYF********,VLL_LSY,HLA-A*02:01,"[[0.945, 0.828, 1.299, -0.169, 0.933, -1.239, ...","[[-0.384, 1.652, 1.33, 1.045, 2.064, 0.26, 0.8...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -1.019...","[[-1.019, -0.987, -1.505, 1.266, -0.912, -0.38...","[[-1.006, -0.59, 1.891, -0.397, 0.412, 1.538, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
2,ATGYPS*,ATKADDK*,CALTVSYGGSQGNLIF****,KGHSH*,LQKENI*,CASSPFSIGQGLTNNEKLFF*,VLL_LSY,HLA-A*02:01,"[[-0.591, -1.302, -0.733, 1.57, -0.146, -0.032...","[[-0.591, -1.302, -0.733, 1.57, -0.146, -0.032...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[1.831, -0.561, 0.533, -0.277, 1.648, -0.384,...","[[-1.019, -0.987, -1.505, 1.266, -0.912, 0.931...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
3,TSGFNG*,NVLDGL**,CAVKMNTGFQKLVF******,DFQATT,SNEGSKA,CSAKLRGSNQPQHF*******,SLM_IPH,HLA-A*02:01,"[[-0.032, 0.326, 2.213, 0.908, 1.313, -0.228, ...","[[0.945, 0.828, 1.299, -0.169, 0.933, -1.337, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[1.05, 0.302, -3.656, -0.259, -3.242, -1.006,...","[[-0.228, 1.399, -4.76, 0.67, -2.647, 0.945, 0...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.228..."
4,MNHNS**,SASEGT**,CASTLRAGWDEQFF******,MNHEY*,SMNVEV*,CASSLSPGGRSEAFF******,VLL_LSY,HLA-A*02:01,"[[-0.663, -1.524, 2.219, -1.005, 1.212, 0.945,...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.591, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-0.663, -1.524, 2.219, -1.005, 1.212, 0.945,...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.663, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,NSMFDY*,ISSIKDK*,CAAAYYTGANSKLTF*****,SGHAT*,FQNNGV*,CASSSPPGVGNEQFF******,VVV_DVG,HLA-A*11:01,"[[0.945, 0.828, 1.299, -0.169, 0.933, -0.228, ...","[[-1.239, -0.547, 2.131, 0.393, 0.816, -0.228,...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.384, ...","[[-1.006, -0.59, 1.891, -0.397, 0.412, 0.931, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
882,NSMFDY*,ISSIKDK*,CAASGTGTASKLTF******,SGHNS*,FNNNVP*,CASETSNNEQFF*********,VVV_DVG,HLA-A*11:01,"[[0.945, 0.828, 1.299, -0.169, 0.933, -0.228, ...","[[-1.239, -0.547, 2.131, 0.393, 0.816, -0.228,...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.384, ...","[[-1.006, -0.59, 1.891, -0.397, 0.412, 0.945, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
883,SSNFYA*,MTLNGDE*,CAFYGSARQLTF********,PRHDT*,FYEKMQ*,CASSPPTTAANYGYTF*****,VVV_DVG,HLA-A*11:01,"[[-0.228, 1.399, -4.76, 0.67, -2.647, -0.228, ...","[[-0.663, -1.524, 2.219, -1.005, 1.212, -0.032...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[0.189, 2.081, -1.628, 0.421, -1.392, 1.538, ...","[[-1.006, -0.59, 1.891, -0.397, 0.412, 0.26, 0...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."
884,DSAIYN*,IQSSQRE*,CAVRGNNNARLMF*******,GTSNPN,SVGIG**,CAWSPGRSPLHF*********,VVV_DVG,HLA-A*11:01,"[[1.05, 0.302, -3.656, -0.259, -3.242, -0.228,...","[[-1.239, -0.547, 2.131, 0.393, 0.816, 0.931, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591...","[[-0.384, 1.652, 1.33, 1.045, 2.064, -0.032, 0...","[[-0.228, 1.399, -4.76, 0.67, -2.647, -1.337, ...","[[-1.343, 0.465, -0.862, -1.02, -0.255, -0.591..."


In [37]:
# convert the encoding result to input feature matrix
X_feature = np.zeros((dataset.shape[0], 0))
for seq in seq_list:
    col_name = seq + "_encode"
    col_feature = np.zeros((0, len_map[seq]*5))
    for i in range(dataset.shape[0]):
        col_feature = np.vstack((col_feature, dataset.loc[i, col_name].reshape(1,-1)))
    X_feature = np.hstack((X_feature, col_feature))
X_feature.shape

(886, 345)

In [42]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# one-hot encode Neo
X_neo = dataset["Neo"].values.reshape(-1,1)
onehotEncoder = OneHotEncoder()
X_neo = onehotEncoder.fit_transform(X_neo).toarray()

# one-hot encode HLA
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(dataset["HLA"].values.reshape(-1,1))

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
# y.shape, X_neo.shape, X_feature.shape
# concat X_neo, X_feature, y to form a matrix


((886,), (886, 11), (886, 345))

### Define the Dataset

In [8]:
def encode_seqCDR(seqCDR):
    encoding_list = []
    for i in range(len(seqCDR)):
        if seqCDR[i] == "*":
            encoding_list.append(np.zeros(5).reshape(1,5))
        else:
            encoding_list.append(af.loc[seqCDR[i]].values.reshape(1,5))
    return np.array(encoding_list).reshape(1,-1)

af = pd.read_csv("~/data/project/pMHC-TCR/library/Atchley_factors.csv")
af.index = af["Amino acid"]
af.drop(columns=["Amino acid"], inplace=True)
af

# torch dataset
class TCRDataset(Dataset):
    '''
    Use each two rows of data as a sample (one for alpha chain, one for beta chain), each sample has the common cellname
    Different from the chain, the HLA class is the same for each sample
    The aaSeqCDR1, aaSeqCDR2, aaSeqCDR3 are the CDR1, CDR2, CDR3 of the alpha chain and beta chain, respectively.
    '''
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        df["Neo"] = df["NeoAA"].str.slice(0,3) + "_" + df["NeoAA"].str.slice(-4,-1)
        df.drop(columns=["NeoAA"], inplace=True)
        for chain in ["AseqCDR", "BseqCDR"]:
            df[chain+"_1"] = df[chain].str.split("_").str[0]
            df[chain+"_2"] = df[chain].str.split("_").str[1]
            df[chain+"_3"] = df[chain].str.split("_").str[2]
            df.drop(columns=[chain], inplace=True)
        seq_list = ["AseqCDR_1", "AseqCDR_2", "AseqCDR_3", "BseqCDR_1", "BseqCDR_2", "BseqCDR_3"]
        # Find the longest sequence in each CDR
        len_map = df[seq_list].applymap(len).max()
        for column in seq_list:
            df[column] = df[column].str.ljust(len_map[column], "*")
        dataset = df[['AseqCDR_1', 'AseqCDR_2', 'AseqCDR_3', 'BseqCDR_1', 'BseqCDR_2', 'BseqCDR_3', 'Neo', 'HLA']]
        for seq in seq_list:
            encode_seq_result = list()
            for i in dataset[seq]:
                encode_seq_result.append(encode_seqCDR(i))
            col_name = seq + "_encode"
            dataset[col_name] = encode_seq_result
        X_feature = np.zeros((dataset.shape[0], 0))
        for seq in seq_list:
            col_name = seq + "_encode"
            col_feature = np.zeros((0, len_map[seq]*5))
            for i in range(dataset.shape[0]):
                col_feature = np.vstack((col_feature, dataset.loc[i, col_name].reshape(1,-1)))
            X_feature = np.hstack((X_feature, col_feature))

        # one-hot encode Neo
        X_neo = dataset["Neo"].values.reshape(-1,1)
        onehotEncoder = OneHotEncoder()
        X_neo = onehotEncoder.fit_transform(X_neo).toarray()

        # one-hot encode HLA
        labelencoder = LabelEncoder()
        y = labelencoder.fit_transform(dataset["HLA"].values.reshape(-1,1))

        # get the final feature matrix
        self.features = np.hstack((X_neo, X_feature))
        self.labels = y

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):        
        return self.features[idx], self.labels[idx]

In [9]:
file_path = '/home/wuxinchao/data/project/data/seqData/230215.csv'
TCRData = TCRDataset(file_path)

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# not use
train_loader = DataLoader(dataset=TCRData,
                          batch_size=32,
                          shuffle=True,
                          num_workers=0)

## Define the model

In [19]:
from torch.autograd import Variable
import torch.optim as optim

In [96]:
class pMHC_TCR_model(nn.Module):
    def __init__(self, input_size, batch_size=32 ,hidden_size=5, num_layers=2, device="cpu") -> None:
        super().__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.label = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input: L, input_size
        if self.batch_size==1:
            x = input.float()
            h_0 = Variable(torch.zeros(self.num_layers, self.hidden_size).to(self.device))
            c_0 = Variable(torch.zeros(self.num_layers, self.hidden_size).to(self.device))
            out, (hn, cn) = self.lstm(x, (h_0, c_0))
            pred = self.label(out[-1, :])
        else:
            x = x.view(-1, self.batch_size, self.seq_length).float()
            # h_0: (num_layers * num_directions, batch, hidden_size)
            h_0 = Variable(torch.zeros(self.num_layers * 1, self.batch_size, self.hidden_size).to(self.device))
            c_0 = Variable(torch.zeros(self.num_layers * 1, self.batch_size, self.hidden_size).to(self.device))
            out, (hn, cn) = self.lstm(x, (h_0, c_0))
            pred = self.label(out[-1, :, :])
        return pred

In [14]:
# not work, for test
a = torch.randn(10, 32, 5)
b = torch.randn(10, 32, 1)
model = nn.Linear(5, 1)
# model(a[-1, :, :]).shape
(b - model(a[-1, :, :])).shape

torch.Size([10, 32, 1])

# Create train and test methods

In [97]:
def train(fold, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = output.view(-1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Fold/Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                fold, epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(fold, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = output.view(-1)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.sigmoid().round()  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set for fold {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\
    '.format(fold, test_loss, correct, len(test_loader.dataset),  100. * correct / len(test_loader.dataset)))

## Create cross validation for training and testing

In [98]:
from sklearn.model_selection import RepeatedKFold, KFold, StratifiedKFold
from torch.utils.data import WeightedRandomSampler

In [100]:
batch_size = 1
seq_length = 2
folds = 5
repeats = 12
epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using {device} for training the network")

def reset_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

model = pMHC_TCR_model(input_size=356, batch_size=batch_size, device=device).to(device)

# kf = RepeatedKFold(n_splits=10, n_repeats=12, random_state=1234)
# kf = KFold(n_splits=10, random_state=1234, shuffle=True)
kf = StratifiedKFold(n_splits=2, random_state=1234, shuffle=True)
weights = torch.FloatTensor([1, 9])
optimizer = optim.Adadelta(model.parameters())

for fold, (train_idx, test_idx) in enumerate(kf.split(TCRData.features, TCRData.labels)):
    print(f"-------------------Fold {fold}-------------------")
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    # train_subsampler = WeightedRandomSampler(weights, len(train_idx), replacement=True)
    # test_subsampler = WeightedRandomSampler(weights, len(test_idx), replacement=True)
    train_loader = torch.utils.data.DataLoader(TCRData, batch_size=batch_size, sampler=train_subsampler)
    test_loader = torch.utils.data.DataLoader(TCRData, batch_size=batch_size, sampler=test_subsampler)
    model.apply(reset_weights)

    for epoch in range(1, epochs + 1):
        train(fold, model, device, train_loader, optimizer, epoch)
        test(fold, model, device, test_loader)

Using cpu for training the network
-------------------Fold 0-------------------


IndexError: too many indices for tensor of dimension 2

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=1234, shuffle=True)

cuda:0


In [267]:
rnn = nn.LSTM(10, 20, 2)
# LSTM: input_size, hidden_size, num_layers
input = torch.randn(5, 3, 10)
# input: tensor of shape (L,H_in) for unbatched input, (L,N,H_in)
h0 = torch.randn(2, 3, 20)
# h0: tensor of shape (num_layers * num_directions, batch, hidden_size)
c0 = torch.randn(2, 3, 20)
# c0: tensor of shape (num_layers * num_directions, batch, hidden_size)
output, (hn, cn) = rnn(input, (h0, c0))

rnn(torch.randn(8, 3, 10), (h0, c0))

In [27]:
# not used
import wandb

wandb.login()
# wandb.init(project="my-awesome-project")
# wandb.init(project="pMHC-TCR", entity="wuxinchao",
#            config={
#                "embedding_length": 5,
#                "hidden_size": 5,
#                "num_layers": 2,
#                "lr": 0.001,
#                "epochs": 100,
#             })
# wandb.watch(model)

# model = pMHC_TCR_model(embedding_length=5, hidden_size=5, num_layers=2)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# wandb.watch(model, log_freq=100)

# model.train()
# for batch_idx, (data, target) in enumerate(train_loader):
#     output = model(data)
#     loss = F.nll_loss(output, target)
#     loss.backward()
#     optimizer.step()
#     if batch_idx % 100 == 0:
#         wandb.log({"loss": loss})

# wandb.finish()